<a href="https://colab.research.google.com/github/Eupene/Data_science/blob/main/car_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pip install libraries that are not available by default

In [ ]:
# pip install libraries that are not available by default
!pip uninstall -y markupsafe jinja2 ydata-profiling
!pip install markupsafe==2.0.1 jinja2==3.0.3
!pip install ydata-profiling
!pip install py-AutoClean

Found existing installation: MarkupSafe 3.0.2
Uninstalling MarkupSafe-3.0.2:
  Successfully uninstalled MarkupSafe-3.0.2
Found existing installation: Jinja2 3.1.5
Uninstalling Jinja2-3.1.5:
  Successfully uninstalled Jinja2-3.1.5
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 4.2 MB/s eta 0:00:00
  Created wheel for markupsafe: filename=MarkupSafe-2.0.1-cp311-cp311-linux_x86_64.whl size=27991 sha256=4b1086192d7320ccad9e488f12de20c3ee859835bb968fd8b13137ad51cbcf14
  Stored in directory: /root/.cache/pip/wheels/ea/18/79/6266ea508b8164a77b95aa19534c77eb805f2878612c37efca
Successfully built markupsafe
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
werkzeug 3.1.3 requires MarkupSafe>=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
flask 3.1.0 requires Jinja2>=3.1.2, but you have jinja2 3.0.3 which is 

#import neccesary libraries

In [ ]:
#import neccesary libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from ydata_profiling import ProfileReport
from AutoClean import AutoClean

#upload dataset

In [ ]:
#upload data
df = pd.read_csv('car data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


#Perform exploratory data analysis (EDA)

EDA helps understand the dataset by identifying trends, patterns, and potential issues. In this project, EDA is used to:

- Check Missing Values – Identify and handle null values.
- Visualize Distributions – Understand price variations and feature relationships.
- Detect Outliers – Remove anomalies that may affect predictions.
- Feature Correlation – Find key factors influencing car prices.

This step ensures clean, meaningful data for better model accuracy.

In [ ]:
# Generates an automated interactive report
profile_eda = ProfileReport(df, explorative=True)

# Displays the report inside the Notebook
profile_eda.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# save the report as an HTML file
profile_eda.to_file("eda_report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

#Data Cleaning

Data cleaning ensures the dataset is accurate and ready for modeling. From the EDA, some duplicate records were found and dropped to avoid bias. Additionally, feature engineering  was also performed such as encoding objects to numericis to improve model performance.

In [ ]:
#Automatically clean the dataset
cleaned_df = AutoClean(df, verbose=True)

23-02-2025 07:31:52.36 - INFO - Started validation of input parameters...
23-02-2025 07:31:52.37 - INFO - Completed validation of input parameters
23-02-2025 07:31:52.37 - INFO - Started handling of duplicates... Method: "AUTO"
23-02-2025 07:31:52.38 - DEBUG - Deletion of 2 duplicate(s) succeeded
23-02-2025 07:31:52.38 - INFO - Completed handling of duplicates in 0.006038 seconds
23-02-2025 07:31:52.38 - INFO - Started handling of missing values...
23-02-2025 07:31:52.38 - DEBUG - 0 missing values found
23-02-2025 07:31:52.38 - INFO - Completed handling of missing values in 0.003312 seconds
23-02-2025 07:31:52.38 - INFO - Started handling of outliers... Method: "WINZ"
23-02-2025 07:31:52.40 - DEBUG - Outlier imputation of 7 value(s) succeeded for feature "Year"
23-02-2025 07:31:52.42 - DEBUG - Outlier imputation of 16 value(s) succeeded for feature "Selling_Price"
23-02-2025 07:31:52.43 - DEBUG - Outlier imputation of 14 value(s) succeeded for feature "Present_Price"
23-02-2025 07:31:5

Logfile saved to: /content/autoclean.log


In [ ]:
cleaned_df.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Transmission_Automatic,Transmission_Manual,Selling_type_Dealer,Selling_type_Individual,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0,False,True,True,False,False,False,True
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0,False,True,True,False,False,True,False
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0,False,True,True,False,False,False,True
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0,False,True,True,False,False,False,True
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0,False,True,True,False,False,True,False


In [ ]:
print(cleaned_df.columns)

Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Driven_kms',
       'Fuel_Type', 'Selling_type', 'Transmission', 'Owner',
       'Transmission_Automatic', 'Transmission_Manual', 'Selling_type_Dealer',
       'Selling_type_Individual', 'Fuel_Type_CNG', 'Fuel_Type_Diesel',
       'Fuel_Type_Petrol'],
      dtype='object')


In [ ]:
print(cleaned_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Car_Name                 299 non-null    object 
 1   Year                     299 non-null    Int64  
 2   Selling_Price            299 non-null    float64
 3   Present_Price            299 non-null    float64
 4   Driven_kms               299 non-null    Int64  
 5   Fuel_Type                299 non-null    object 
 6   Selling_type             299 non-null    object 
 7   Transmission             299 non-null    object 
 8   Owner                    299 non-null    Int64  
 9   Transmission_Automatic   299 non-null    bool   
 10  Transmission_Manual      299 non-null    bool   
 11  Selling_type_Dealer      299 non-null    bool   
 12  Selling_type_Individual  299 non-null    bool   
 13  Fuel_Type_CNG            299 non-null    bool   
 14  Fuel_Type_Diesel         2

#Define Features and Labels

##Feature Selection for Car Price Prediction

The features for the car price prediction model were selected based on their relevance to pricing.

##Target Variable (Label)
Selling_Price: This is the dependent variable we aim to predict.
Selected Features

##The following features were chosen based on their impact on car prices:

Year: Helps determine the car’s age, which affects its value.
Present_Price: Represents the original price of the car, influencing its resale value.
Driven_kms: Indicates car usage; higher mileage usually lowers price.
Owner: More previous owners may reduce the price.
Transmission_Automatic, Transmission_Manual: Transmission type can impact pricing.
Selling_type_Dealer, Selling_type_Individual: Dealers and individual sellers may have different pricing strategies.
Fuel_Type_CNG, Fuel_Type_Diesel, Fuel_Type_Petrol: Fuel type affects demand and price.

##Dropped Features and Reasons
Car_Name: Each car model has a unique name, making it unsuitable as a predictive feature.
Fuel_Type, Selling_type, Transmission: These categorical columns were replaced with their one-hot encoded versions to improve model performance.

In [ ]:
#define your features and lables

#Selected features
X = cleaned_df[["Year", "Present_Price", "Driven_kms", "Owner",
        "Transmission_Automatic", "Transmission_Manual",
        "Selling_type_Dealer", "Selling_type_Individual",
        "Fuel_Type_CNG", "Fuel_Type_Diesel", "Fuel_Type_Petrol"]]
#Target Variable (Label)
y = cleaned_df["Selling_Price"]

#Split dataset into train and test data

##Train-Test Split Process
- The dataset was split into training and testing sets to evaluate the model’s performance. The training set (80%) is used to train the model, while the test set (20%) is used to assess its accuracy on unseen data.

- The selected features (X) include numerical attributes such as Year, Present Price, Driven Kilometers, and Owner, along with categorical variables like Fuel Type, Selling Type, and Transmission, which were one-hot encoded. These features were chosen based on their potential impact on car prices.

- The target variable (y) is Selling Price, as the goal is to predict the price of a car.

- A random state was set to ensure reproducibility, and an 80-20 split was used to maintain a balance between training and evaluation while preventing overfitting

In [ ]:
#now split the data into 80 training and 20 testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#now split the data into 80 training and 20 testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#print the number of rows in each set
print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))

Training set size: 239
Testing set size: 60


#Model Training

- Linear Regression: A simple model that finds a straight-line relationship between features and the target variable. It works well when there is a linear correlation between inputs and outputs.

- Decision Tree: A model that splits the data into branches based on conditions. It is prone to overfitting, leading to lower accuracy on unseen data.

- Random Forest: An improvement over Decision Trees, it uses multiple trees to reduce overfitting and improve accuracy.

- Gradient Boosting: A sequential learning model that improves predictions by correcting previous errors. It is effective in handling complex relationships.

- XGBoost: An optimized version of Gradient Boosting, known for its efficiency and speed. It performs well in structured data problems.

## Determining the Best Model
The best model is selected based on R² Score and error metrics (MAE, MSE).

R² Score (Coefficient of Determination): Measures how well the model explains the variance in the target variable.

- Above 0.70 → Good fit
- 0.50 - 0.70 → Moderate fit
- Below 0.50 → Poor fit
- Lowest MAE and MSE indicate fewer prediction errors.


##Model Training Process
The model training process involved selecting multiple regression models to predict car prices. The dataset was first split into training (80%) and testing (20%) sets to ensure fair evaluation. Various models were trained, including Linear Regression, Decision Tree, Random Forest, Gradient Boosting, and XGBoost. Each model was fitted using the training data to learn patterns and relationships between the features and the target variable (Selling Price). The purpose of training multiple models was to compare their performance and determine the most accurate and reliable one for price prediction.



In [ ]:
# Initialize models
lr_model = LinearRegression()
dt_model = DecisionTreeRegressor(random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Train models
lr_model.fit(X_train, y_train)
dt_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

#Evaluate Model

##Model Evaluation Process
After training, the models were evaluated using the test dataset to measure their performance. Three key metrics were used:

- Mean Absolute Error (MAE): Measures the average absolute differences between predicted and actual prices.
- Mean Squared Error (MSE): Captures the average squared differences, penalizing larger errors.
- R² Score: Indicates how well the model explains the variance in car prices.
- By comparing these metrics across all models, the best-performing one can be selected for final deployment. A lower MAE and MSE, along with a higher R² score, indicate a better model.









In [ ]:
# Function to evaluate models
def evaluate_model(model, name):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"{name} Performance:")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"R² Score: {r2:.4f}")
    print("-" * 50)

# Evaluate all models
evaluate_model(lr_model, "Linear Regression")
evaluate_model(dt_model, "Decision Tree")
evaluate_model(rf_model, "Random Forest")
evaluate_model(gb_model, "Gradient Boosting")
evaluate_model(xgb_model, "XGBoost")

Linear Regression Performance:
Mean Absolute Error (MAE): 1.2436
Mean Squared Error (MSE): 3.7609
R² Score: 0.7240
--------------------------------------------------
Decision Tree Performance:
Mean Absolute Error (MAE): 1.1875
Mean Squared Error (MSE): 6.2427
R² Score: 0.5418
--------------------------------------------------
Random Forest Performance:
Mean Absolute Error (MAE): 1.0225
Mean Squared Error (MSE): 4.8106
R² Score: 0.6469
--------------------------------------------------
Gradient Boosting Performance:
Mean Absolute Error (MAE): 0.9775
Mean Squared Error (MSE): 4.1923
R² Score: 0.6923
--------------------------------------------------
XGBoost Performance:
Mean Absolute Error (MAE): 0.9528
Mean Squared Error (MSE): 4.7823
R² Score: 0.6490
--------------------------------------------------


#Model Performance Analysis and Recommendation
The evaluation results indicate that Gradient Boosting and XGBoost performed the best among the models tested.

- Linear Regression achieved a decent R² score (0.7240) but had relatively high MAE (1.2436) and MSE (3.7609), indicating some prediction errors.
- Decision Tree performed the worst, with the lowest R² score (0.5418) and highest MSE (6.2427), making it unreliable.
- Random Forest improved over the Decision Tree but still had a lower R² score (0.6469).
- Gradient Boosting provided the best balance, with the lowest MAE (0.9775) and a strong R² score (0.6923).
- XGBoost performed similarly to Gradient Boosting but had a slightly lower R² score (0.6490).

##Recommendation
Based on these results, Gradient Boosting is the best model for car price prediction, as it balances accuracy and error reduction. However, further optimization using hyperparameter tuning and feature engineering could improve performance. It is recommended to fine-tune the Gradient Boosting model and explore additional techniques such as cross-validation to enhance predictions.